In [1]:
import numpy as np

I want to experimentally prove that the supposedly right answer of the Monty Hall Problem is incorrect.
The game goes as follows: You are confronted with 3 doors, behind of which one is a car (WIN).
The other ones have goats/nothing (LOOSE).
You only have one choice. So we start with a clear 1/3 probability of winning for each door.

After you make your choice, the game host (knowing the location of the WIN) opens one door that contains a LOOSE and offers you to change.
Appearently according to statistical definitions the probabilities have locked in after your choice and the 1/3 WIN probability falls to the other door you did not choose (as your door must somehow stay at 1/3 chance).
This gives the door you have currently not selected a 2/3 chance of winnig and you should definetly switch.
I personally think that is an error in the definition of statistics. The situation has changed and the winning probability should now be 1/2 for each of the two remaining doors.
How is that scenario different from starting with two doors in the first place?

So, in this jupyter notebook I will simulate multiple runs of this game first for one, then for a second player.
Player Math will always switch doors when offered the choice.
Player Me will always stay at his selected door.
If those statistical definitions represent reality Player Math should win 2/3 of his games and player Me 1/3.
If I am right both players Math and player Me should even out at a 50/50 win chance, showing that this choice has no effect.                                           

In [2]:
np.random.seed() #true random numbers https://docs.scipy.org/doc/numpy-1.3.x/reference/generated/numpy.random.seed.html

def Math(player_d, host_d):
    player_d += 1 #choose the other closed door (try the one next to it)
    if player_d == 4:
        player_d = 1 #modulo would fuck up here
    if player_d == host_d: #if the next door is already the one opened, go one door further, reaching the other closed door for sure
        player_d += 1
    if player_d == 4:
        player_d = 1 #modulo would fuck up here
    return player_d

def Me(player_d, host_d):
    return player_d

player_wins = 0
player_looses = 0
for i in range(100000):
    win_door = np.random.randint(1, 3+1) #the door that contains the WIN (shall not be seen by player)
    player_door = np.random.randint(1, 3+1) #the door the player chooses initially
    host_door = np.random.randint(1, 3+1) #the door the host opens, containing a known LOOSE
    while host_door == win_door or host_door == player_door:  #make sure the host doesn't reaveal a WIN and does not open the players door
        host_door = np.random.randint(1, 3+1)
    #the player now gets access to the door the host has opened as a LOOSE (never the player door) and can change his choice
    # ====== choose your player =======
    #player_door = Me(player_door, host_door)
    player_door = Math(player_door, host_door)
    # =================================
    if player_door == win_door:
        player_wins += 1
    elif player_door == host_door:
        print("internal error")
    else:
        player_looses += 1
    #print(win_door, player_door, host_door, player_wins, player_looses)
print("Player won " + str(player_wins) + " out of " + str(player_wins+player_looses) + " Monty Hall games")

Player won 66808 out of 100000 Monty Hall games


I was wrong. Math wins 2/3 of the games while Me wins 1/3 of the games.

This was my first implementation of the host door:

In [ ]:
    host_door = np.random.randint(1, 3+1) #the door the host opens, containing a known LOOSE
    while host_door == win_door: #make sure the host doesn't reaveal a WIN
        host_door = np.random.randint(1, 3+1)
    #the player now gets access to the door the host has revealed as a LOOSE and can change his choice
    

But as I now realize during the implementation, the host doesn't just randomly open one of the two LOOSE doors, but specifically the one you have not yet opened (never opening your door - that would make the "do you want to change?" choice pretty obvious). So the correct implementation would be as below.
It could be that kind of selectiveness that changes the odds of the situation.

In [ ]:
    host_door = np.random.randint(1, 3+1) #the door the host opens, containing a known LOOSE
    while host_door == win_door or host_door == player_door: #make sure the host doesn't reaveal a WIN and does not open the players door
        host_door = np.random.randint(1, 3+1)
    #the player now gets access to the door the host has revealed as a LOOSE (never the player door) and can change his choice
    

To explain this intuitively to myself:

The most likely scenario is that you choose a LOOSE door at the start (with 2/3). So if we assume that is the case, then the host would eliminate the other LOOSE door for you, giving you a free win in this scenario.
It is only in the 1/3 chance that you choose the WIN at the start, that you don't get any help by the elimination and the continuation of that changing doors strategy would guarantee you a loose. Thus we end up winning 2/3.

Whereas staying at your previous door ignores the additional information and just gives you the starting odds of 1/3.

Let's try the first, wrong implementation to confirm that this difference was the turning point (host can open the players door, the player will obviously open a different one if he does so):

In [3]:
def Me(player_d, host_d):
    if player_d == host_d:      #even I would switch if I saw that the LOOSE is in front of me
        player_d += 1
    if player_d == 4:
        player_d = 1
    return player_d


#wrong implementation: 
player_wins = 0
player_looses = 0
for i in range(100000):
    win_door = np.random.randint(1, 3+1) #the door that contains the WIN (shall not be seen by player)
    player_door = np.random.randint(1, 3+1) #the door the player chooses initially
    host_door = np.random.randint(1, 3+1) #the door the host opens, containing a known LOOSE
    while host_door == win_door: #make sure the host doesn't reaveal a WIN or select the player_door (would defy purpose)
        host_door = np.random.randint(1, 3+1)
    #the player now gets access to the door the host has opened as a LOOSE (never the player door) and can change his choice
    # ====== choose your player =======
    #player_door = Me(player_door, host_door)
    player_door = Math(player_door, host_door)
    # ==================================
    if player_door == win_door:
        player_wins += 1
    elif player_door == host_door:
        print("internal error")
    else:
        player_looses += 1
    #print(win_door, player_door, host_door, player_wins, player_looses)
print("Player won " + str(player_wins) + " out of " + str(player_wins+player_looses) + " 1/2 games")

Player won 49914 out of 100000 1/2 games


Indeed. We are back at 50/50 for both players. So it is the fact that the host selectively chooses a LOOSE door THAT IS NOT YOUR SELECTED DOOR, that reveals additional information that can be effectively harnessed with the door-changing strategy.

In [ ]:
Hmm. I proofed math. v